In [1]:
import numpy as np
import pandas as pd

In [10]:
movies_df=pd.read_csv(r"C:\Users\Vishruta\OneDrive\Desktop\movie_reccomendation_system\movies.csv",usecols=['movieId','title'],dtype={'movieId':'int32','title':'str'})
ratings_df=pd.read_csv(r"C:\Users\Vishruta\OneDrive\Desktop\movie_reccomendation_system\ratings.csv",usecols=['userId','movieId','rating'],dtype={'userId':'int32',"movieId":"int32","rating":"float32"})
movies_df.tail()


,movieId,title
9737,193581,Black Butler: Book of the Atlantic (2017)
9738,193583,No Game No Life: Zero (2017)
9739,193585,Flint (2017)
9740,193587,Bungo Stray Dogs: Dead Apple (2018)
9741,193609,Andrew Dice Clay: Dice Rules (1991)


In [11]:
ratings_df.tail()


,userId,movieId,rating
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0
100835,610,170875,3.0


In [14]:
data=pd.merge(ratings_df,movies_df,on="movieId")
data.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"


In [18]:
drop_data=data.dropna(axis=0,subset=['title'])
drop_data

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"
...,...,...,...,...
100831,610,166534,4.0,Split (2017)
100832,610,168248,5.0,John Wick: Chapter Two (2017)
100833,610,168250,5.0,Get Out (2017)
100834,610,168252,5.0,Logan (2017)


In [21]:
renamed_count=(drop_data.groupby(by=['title'])['rating'].count().reset_index().rename(columns={'rating':"totalRatingCount"})[['title','totalRatingCount']])
renamed_count.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [24]:
merged_data=drop_data.merge(renamed_count,left_on="title",right_on="title",how="left")
merged_data.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,1,3,4.0,Grumpier Old Men (1995),52
2,1,6,4.0,Heat (1995),102
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.0,"Usual Suspects, The (1995)",204


In [26]:
print(merged_data.describe())

              userId        movieId         rating  totalRatingCount
count  100836.000000  100836.000000  100836.000000     100836.000000
mean      326.127564   19435.295718       3.501557         58.758777
std       182.618491   35530.987199       1.042541         61.965384
min         1.000000       1.000000       0.500000          1.000000
25%       177.000000    1199.000000       3.000000         13.000000
50%       325.000000    2991.000000       3.500000         39.000000
75%       477.000000    8122.000000       4.000000         84.000000
max       610.000000  193609.000000       5.000000        329.000000


In [28]:
rating_threshold=60
recommendation_rule=merged_data.query('totalRatingCount>=@rating_threshold')
recommendation_rule.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
2,1,6,4.0,Heat (1995),102
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.0,"Usual Suspects, The (1995)",204
7,1,110,4.0,Braveheart (1995),237


In [31]:
movie_features=recommendation_rule.pivot_table(index="title",columns='userId',values="rating").fillna(0)
movie_features

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
"40-Year-Old Virgin, The (2005)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
"Abyss, The (1989)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0
Ace Ventura: Pet Detective (1994),0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,2.0,0.0,0.0,0.0,3.5,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Willy Wonka & the Chocolate Factory (1971),5.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,4.0,0.0,0.0,3.5,0.0,0.0
"Wizard of Oz, The (1939)",5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,2.0,0.0,5.0,2.5,0.0,3.5
X-Men (2000),5.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,3.5


In [35]:
from scipy.sparse import csr_matrix
movie_array_matrix=csr_matrix(movie_features.values)
movie_array_matrix


<335x603 sparse matrix of type '<class 'numpy.float32'>'
	with 35080 stored elements in Compressed Sparse Row format>

In [37]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(metric="cosine",algorithm="brute")
model.fit(movie_array_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [40]:
#from the moive indices,select a random index as test record
test_record=np.random.choice(movie_array_matrix.shape[0])
print(test_record)


270


In [48]:
#1st recommended movie will be itself
distances,indices=model.kneighbors(movie_features.iloc[test_record,:].values.reshape(1,-1),n_neighbors=11)
movie_features.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
"40-Year-Old Virgin, The (2005)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
"Abyss, The (1989)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0
Ace Ventura: Pet Detective (1994),0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,2.0,0.0,0.0,0.0,3.5,0.0,3.0


In [50]:
for i in range(0,len(distances.reshape(-1))):
    if(i==0):
        print(f"top 10 movies recommended for {movie_features.index[test_record]} are:")
    else:
        print(f"{i} : {movie_features.index[indices.reshape(-1)[i]]} with the distance of {distances.reshape(-1)[i]}")


top 10 movies recommended for Silence of the Lambs, The (1991) are:
1 : Pulp Fiction (1994) with the distance of 0.2906176447868347
2 : Shawshank Redemption, The (1994) with the distance of 0.35293447971343994
3 : Forrest Gump (1994) with the distance of 0.3605371117591858
4 : Seven (a.k.a. Se7en) (1995) with the distance of 0.36901718378067017
5 : Fugitive, The (1993) with the distance of 0.4056590795516968
6 : Schindler's List (1993) with the distance of 0.40848058462142944
7 : Jurassic Park (1993) with the distance of 0.41301190853118896
8 : Fargo (1996) with the distance of 0.41899752616882324
9 : Terminator 2: Judgment Day (1991) with the distance of 0.42032676935195923
10 : Usual Suspects, The (1995) with the distance of 0.4252413511276245
